In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:

import os
import glob
import sys
import multiprocessing as mpp

reduce_factor=2
new_size=400
delay=20
max_frame = 500

mpp_mode=True

### Earthquake version:
(wrap in function calls so we can mpp them).

In [15]:
def do_eq():
    files_list = sorted([g for j,g in enumerate(glob.glob('imagesEq/*.png')) if int(g.split('-')[1].split('.')[0])<max_frame],
                        key=lambda rw: int(rw.split('-')[1].split('.')[0]))[::reduce_factor]
    print(files_list[0:5])


    convert_str = 'convert -delay {} -loop 0 -resize {}x {} animated_dwf_eq.gif'.format(delay, new_size, 
        ' '.join(files_list)) 

    #print(convert_str)
    os.system(convert_str)

### Econ version:

In [16]:
def do_econ():
    files_list = sorted([g for j,g in enumerate(glob.glob('imagesEcon/*.png')) if int(g.split('-')[1].split('.')[0])<max_frame],
                        key=lambda rw: int(rw.split('-')[1].split('.')[0]))[::reduce_factor]
    print(files_list[0:5])


    convert_str = 'convert -delay {} -loop 0 -resize {}x {} animated_dwf_econ.gif'.format(delay, new_size, 
        ' '.join(files_list)) 

    #print(convert_str)
    os.system(convert_str)

### MPP (python) runner:
(actaully, i think "convert" is multi-threaded, so this doesn't really do anything for us.

In [17]:
if mpp_mode:
    # ... and if this gets any more complicated, we'll want a Pool() or something...
    # and note that it is possible that this will go badly becaust the matplotlib backents might interfere.
    # (except this scritp assumes mpl is already done... which means, i think, that "convert" is already threaded, right?
    # so we don't need to mpp here?)
    #
    P1 = mpp.Process(target=do_eq)
    P2 = mpp.Process(target=do_econ)

    P1.start()
    print('P1 started.')
    P2.start()
    print('P2 started.')
    #
    # they don't return anything, so we don't really need to join, but it won't hurt...
    P1.join()
    print('finished P1 join()')
    P2.join()
    print('finishde P2 join()')

    print('we should be finished...')



P1 started.
['imagesEcon/dwaveFrame-0000.png', 'imagesEcon/dwaveFrame-0002.png', 'imagesEcon/dwaveFrame-0004.png', 'imagesEcon/dwaveFrame-0006.png', 'imagesEcon/dwaveFrame-0008.png']
['imagesEq/dwaveFrame-0000.png', 'imagesEq/dwaveFrame-0002.png', 'imagesEq/dwaveFrame-0004.png', 'imagesEq/dwaveFrame-0006.png', 'imagesEq/dwaveFrame-0008.png']
P2 started.
finished P1 join()
finishde P2 join()
we should be finished...


### Serial runner
( and i think "convert" does good multi-threading )

In [18]:
if not mpp_mode:
    print('starting...')
    print('doing_eq()')
    a=do_eq()

    print('doing econ()...')
    b=do_econ()
    print('finished.')